In [1]:
import sqlite3 as sql

import pandas as pd

import weather

In [2]:
def add_colon(time_str):
    return (time_str[:-2] + ":" + time_str[-2:]).zfill(4)

In [3]:
flights = pd.read_csv('data/flights_1%.csv', index_col=0)
test = pd.read_csv('data/test.csv', index_col=0)
train = pd.read_csv(
    'data/flights_train.csv',
    index_col=0,
    converters={
        'crs_dep_time': add_colon,
        'crs_arr_time': add_colon,
    },
    parse_dates=[
        ['fl_date', 'crs_dep_time'],
        ['fl_date', 'crs_arr_time']
    ]
).reset_index().set_index('id')

In [4]:
pd.concat([flights.origin, flights.dest]).value_counts().head(100).sum()

288373

In [5]:
# [
#     'K' + code for code in
#     pd.concat([flights.origin, flights.dest]).value_counts().head(100).index
# ]

I used the above to filter the weather data

In [6]:
weather = pd.read_csv('data/weather_events.csv', parse_dates=['start_time_utc', 'end_time_utc'])
weather.start_time_utc = weather.start_time_utc.dt.tz_localize('UTC')
weather.end_time_utc = weather.end_time_utc.dt.tz_localize('UTC')

In [7]:
weather.head()

,event_id,event_type,severity,start_time_utc,end_time_utc,time_zone,airport_code,location_lat,location_lng,city,county,us_state,zip_code,iata_code
0,W-21507,Snow,Light,2017-12-31 13:36:00+00:00,2017-12-31 15:53:00+00:00,US/Eastern,KGRR,42.8808,-85.5228,Grand Rapids,Kent,MI,49512,GRR
1,W-21508,Snow,Light,2018-01-01 01:53:00+00:00,2018-01-01 07:53:00+00:00,US/Eastern,KGRR,42.8808,-85.5228,Grand Rapids,Kent,MI,49512,GRR
2,W-21509,Snow,Light,2018-01-01 11:53:00+00:00,2018-01-01 19:53:00+00:00,US/Eastern,KGRR,42.8808,-85.5228,Grand Rapids,Kent,MI,49512,GRR
3,W-21510,Snow,Light,2018-01-02 00:53:00+00:00,2018-01-02 09:53:00+00:00,US/Eastern,KGRR,42.8808,-85.5228,Grand Rapids,Kent,MI,49512,GRR
4,W-21511,Snow,Light,2018-01-02 10:53:00+00:00,2018-01-02 16:53:00+00:00,US/Eastern,KGRR,42.8808,-85.5228,Grand Rapids,Kent,MI,49512,GRR


In [8]:
weather_local = weather.copy()

In [9]:
weather_local['start_time_local'] = weather_local.groupby(
    'time_zone'
).start_time_utc.transform(
    lambda x: x.dt.tz_convert(x.name).dt.tz_localize(None)
)
weather_local['end_time_local'] = weather_local.groupby(
    'time_zone'
).end_time_utc.transform(
    lambda x: x.dt.tz_convert(x.name).dt.tz_localize(None)
)

In [10]:
pd.crosstab(weather_local.event_type, weather_local.severity)

severity,Heavy,Light,Moderate,Other,Severe,UNK
event_type,,,,,,
Cold,0,0,0,0,413,0
Fog,0,0,7195,0,8633,0
Hail,0,0,0,537,0,0
Precipitation,0,0,0,0,0,2519
Rain,4323,63993,14857,0,0,0
Snow,738,10345,2906,0,0,0
Storm,0,0,0,0,406,0


In [22]:
weather_dummy = weather_local.drop(
    ['event_type', 'severity'], axis=1
).join(
    pd.get_dummies(weather_local.event_type).rename(str.lower, axis=1)
).join(
    weather_local.severity.replace({
        "Light": 1,
        "UNK": 1,
        "Moderate": 2,
        "Heavy": 3,
        "Severe": 3,
        "Other": 3,
    })
)

In [23]:
weather_dummy

,event_id,start_time_utc,end_time_utc,time_zone,airport_code,location_lat,location_lng,city,county,us_state,...,start_time_local,end_time_local,cold,fog,hail,precipitation,rain,snow,storm,severity
0,W-21507,2017-12-31 13:36:00+00:00,2017-12-31 15:53:00+00:00,US/Eastern,KGRR,42.8808,-85.5228,Grand Rapids,Kent,MI,...,2017-12-31 08:36:00,2017-12-31 10:53:00,0,0,0,0,0,1,0,1
1,W-21508,2018-01-01 01:53:00+00:00,2018-01-01 07:53:00+00:00,US/Eastern,KGRR,42.8808,-85.5228,Grand Rapids,Kent,MI,...,2017-12-31 20:53:00,2018-01-01 02:53:00,0,0,0,0,0,1,0,1
2,W-21509,2018-01-01 11:53:00+00:00,2018-01-01 19:53:00+00:00,US/Eastern,KGRR,42.8808,-85.5228,Grand Rapids,Kent,MI,...,2018-01-01 06:53:00,2018-01-01 14:53:00,0,0,0,0,0,1,0,1
3,W-21510,2018-01-02 00:53:00+00:00,2018-01-02 09:53:00+00:00,US/Eastern,KGRR,42.8808,-85.5228,Grand Rapids,Kent,MI,...,2018-01-01 19:53:00,2018-01-02 04:53:00,0,0,0,0,0,1,0,1
4,W-21511,2018-01-02 10:53:00+00:00,2018-01-02 16:53:00+00:00,US/Eastern,KGRR,42.8808,-85.5228,Grand Rapids,Kent,MI,...,2018-01-02 05:53:00,2018-01-02 11:53:00,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116860,W-5874974,2018-02-05 13:19:00+00:00,2018-02-05 13:53:00+00:00,US/Eastern,KTPA,27.9633,-82.5400,Tampa,Hillsborough,FL,...,2018-02-05 08:19:00,2018-02-05 08:53:00,0,1,0,0,0,0,0,3
116861,W-5874975,2018-02-13 13:53:00+00:00,2018-02-13 14:53:00+00:00,US/Eastern,KTPA,27.9633,-82.5400,Tampa,Hillsborough,FL,...,2018-02-13 08:53:00,2018-02-13 09:53:00,0,1,0,0,0,0,0,2
116862,W-5874976,2018-02-15 14:46:00+00:00,2018-02-15 15:53:00+00:00,US/Eastern,KTPA,27.9633,-82.5400,Tampa,Hillsborough,FL,...,2018-02-15 09:46:00,2018-02-15 10:53:00,0,1,0,0,0,0,0,2
116863,W-5874977,2018-02-17 11:53:00+00:00,2018-02-17 12:38:00+00:00,US/Eastern,KTPA,27.9633,-82.5400,Tampa,Hillsborough,FL,...,2018-02-17 06:53:00,2018-02-17 07:38:00,0,1,0,0,0,0,0,3


In [24]:
weather_sel = weather_dummy[[
    'iata_code', 'start_time_local', 'end_time_local',
    'cold', 'fog', 'hail', 'precipitation', 'rain', 'snow', 'storm',
    'severity',
]]

In [14]:
train.head()

,fl_date_crs_dep_time,fl_date_crs_arr_time,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
id,,,,,,,,,,,,,,,,,,,,,
121941,2019-08-25 18:18:00,2019-08-25 19:21:00,AA,AA_CODESHARE,AA,5636,OH,N723PS,5636,11057,...,140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109932,2018-03-02 07:45:00,2018-03-02 09:15:00,WN,WN,WN,2028,WN,N244WN,2028,13495,...,471,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63108,2019-03-06 12:14:00,2019-03-06 15:31:00,UA,UA,UA,545,UA,N69804,545,11292,...,862,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91519,2019-10-11 06:00:00,2019-10-11 08:06:00,DL,DL_CODESHARE,DL,3798,OO,N452SW,3798,16869,...,553,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8361,2019-09-26 14:26:00,2019-09-26 15:21:00,UA,UA_CODESHARE,UA,5359,OO,N120SY,5359,10713,...,522,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
train_sel = train[[
    'origin', 'fl_date_crs_dep_time',
    'dest', 'fl_date_crs_arr_time',
]]

In [35]:
conn = sql.connect(':memory:')
weather_sel.to_sql('weather', conn, index=False)
train_sel.to_sql('flights', conn)
conn.cursor().execute('create index weather_idx on weather (iata_code, start_time_local, end_time_local)')

In [27]:
pd.read_sql_query('select * from flights', conn).head()

,id,origin,fl_date_crs_dep_time,dest,fl_date_crs_arr_time
0,121941,CLT,2019-08-25 18:18:00,AGS,2019-08-25 19:21:00
1,109932,MSY,2018-03-02 07:45:00,BNA,2018-03-02 09:15:00
2,63108,DEN,2019-03-06 12:14:00,IAH,2019-03-06 15:31:00
3,91519,XWA,2019-10-11 06:00:00,MSP,2019-10-11 08:06:00
4,8361,BOI,2019-09-26 14:26:00,SFO,2019-09-26 15:21:00


In [28]:
pd.read_sql_query('select * from weather', conn).head()

,iata_code,start_time_local,end_time_local,cold,fog,hail,precipitation,rain,snow,storm,severity
0,GRR,2017-12-31 08:36:00,2017-12-31 10:53:00,0,0,0,0,0,1,0,1
1,GRR,2017-12-31 20:53:00,2018-01-01 02:53:00,0,0,0,0,0,1,0,1
2,GRR,2018-01-01 06:53:00,2018-01-01 14:53:00,0,0,0,0,0,1,0,1
3,GRR,2018-01-01 19:53:00,2018-01-02 04:53:00,0,0,0,0,0,1,0,1
4,GRR,2018-01-02 05:53:00,2018-01-02 11:53:00,0,0,0,0,0,1,0,1


In [40]:
query = """
select flights.id,
max(wo.cold) as cold_o, max(wo.fog) as fog_o, max(wo.hail) as hail_o,
max(wo.precipitation) as precipitation_o,
max(wo.rain) as rain_o, max(wo.snow) as snow_o, max(wo.storm) as storm_o,
max(wo.severity) as severity_o,
max(wd.cold) as cold_d, max(wd.fog) as fog_d, max(wd.hail) as hail_d,
max(wd.precipitation) as precipitation_d,
max(wd.rain) as rain_d, max(wd.snow) as snow_d, max(wd.storm) as storm_d,
max(wd.severity) as severity_d
from flights
left join weather as wo
on (
    flights.origin = wo.iata_code
    and flights.fl_date_crs_dep_time between
    wo.start_time_local and wo.end_time_local
)
left join weather as wd
on (
    flights.dest = wd.iata_code
    and flights.fl_date_crs_arr_time between
    wd.start_time_local and wd.end_time_local
)
group by flights.id;
"""

train_with_weather = pd.read_sql_query(query, conn).fillna(0)

In [44]:
for col in train_with_weather.columns:
    train_with_weather[col] = train_with_weather[col].astype(int)

In [48]:
train_with_weather = train.join(train_with_weather.set_index('id'))

In [32]:
conn.close()

In [4]:
train_with_weather = weather.add_weather(train)